In [1]:
%pip install -U hats lsdb

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pathlib import Path

import lsdb
from dask.distributed import Client

In [2]:
# "Global" paths on PSC Bridges2 cluster
GLOBAL_HATS_PATH = Path("/ocean/projects/phy210048p/shared/hats/catalogs/")
LOCAL_HATS_PATH = Path("./hats")

PS1_OTMO_PATH = GLOBAL_HATS_PATH / "ps1/ps1_otmo"
PS1_OTMO_MARGIN_PATH = GLOBAL_HATS_PATH / "ps1/ps1_otmo_10arcs"

ZUBERCAL_PATH = GLOBAL_HATS_PATH / "ztf_dr16/zubercal"

CATALOG_TYPE = "vrrlyr"
GAIA_VARS_PATH = LOCAL_HATS_PATH / f"gaia_dr3_{CATALOG_TYPE}"
GAIA_VARS_MARGIN_PATH = LOCAL_HATS_PATH / f"gaia_dr3_{CATALOG_TYPE}_10arcsec"

PS1_FILTERS = 'grizy'
PS1_MAG_SUFFIXES = ['MeanPSFMag', 'MeanPSFMagErr', 'Flags']

In [3]:
%%time

# Few useful columns from PS1 object catalog
ps1_otmo = lsdb.read_hats(
    PS1_OTMO_PATH,
    margin_cache=PS1_OTMO_MARGIN_PATH,
    columns=(
        ['objID', 'raMean', 'decMean']
        + [f'{fltr}{suffix}' for fltr in PS1_FILTERS for suffix in PS1_MAG_SUFFIXES]
    )
)
display(ps1_otmo)

# Zubercal catalog, skip coordinates and few other columns
# Column description:
# http://atua.caltech.edu/ZTF/Fields/ReadMe.txt
zubercal = lsdb.read_hats(
    ZUBERCAL_PATH,
    columns=['mjd', 'mag', 'magerr', 'objectid', 'info', 'flag', 'band'],
)
display(zubercal)

gaia_variables = lsdb.read_hats(
    GAIA_VARS_PATH,
    margin_cache=GAIA_VARS_MARGIN_PATH,
)
display(gaia_variables)

# Combine everything!
matched = gaia_variables.crossmatch(
    ps1_otmo,
    radius_arcsec=1.0,
    suffixes=["", ""],
    output_catalog_name="gaia_vars_x_ps1_otmo",
).join_nested(
    zubercal,
    left_on='objID',
    right_on='objectid',
    nested_column_name='lc',
    output_catalog_name='gaia_vars_x_ps1_otmo_at_zubercal',
)
display(matched)

,objID,raMean,decMean,gMeanPSFMag,gMeanPSFMagErr,gFlags,rMeanPSFMag,rMeanPSFMagErr,rFlags,iMeanPSFMag,iMeanPSFMagErr,iFlags,zMeanPSFMag,zMeanPSFMagErr,zFlags,yMeanPSFMag,yMeanPSFMagErr,yFlags
npartitions=27161,,,,,,,,,,,,,,,,,,
0,int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow]
281474976710656,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458694145076363264,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458764513820540928,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


,mjd,mag,magerr,objectid,info,flag,band
npartitions=70853,,,,,,,
0,double[pyarrow],float[pyarrow],uint16[pyarrow],int64[pyarrow],int32[pyarrow],uint8[pyarrow],string[pyarrow]
281474976710656,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
3458694145076363264,...,...,...,...,...,...,...
3458764513820540928,...,...,...,...,...,...,...


,_RAJ2000,_DEJ2000,recno,Harm,Source,SolID,PF,PF.mask,e_PF,e_PF.mask,P1O,P1O.mask,e_P1O,e_P1O.mask,_tab20_8,e_EpochG,_tab20_10,_tab20_10.mask,e_EpochBP,e_EpochBP.mask,_tab20_12,_tab20_12.mask,e_EpochRP,e_EpochRP.mask,_tab20_14,_tab20_14.mask,e_EpochRV,e_EpochRV.mask,Gmagavg,e_Gmagavg,BPmagavg,BPmagavg.mask,e_BPmagavg,e_BPmagavg.mask,RPmagavg,RPmagavg.mask,e_RPmagavg,e_RPmagavg.mask,RVavg,RVavg.mask,e_RVavg,e_RVavg.mask,ptpG,e_ptpG,ptpBP,ptpBP.mask,e_ptpBP,e_ptpBP.mask,ptpRP,ptpRP.mask,e_ptpRP,e_ptpRP.mask,ptpRV,ptpRV.mask,e_ptpRV,e_ptpRV.mask,__M_H_,__M_H_.mask,e__M_H_,e__M_H_.mask,R21G,R21G.mask,e_R21G,e_R21G.mask,R31G,R31G.mask,e_R31G,e_R31G.mask,phi21G,phi21G.mask,e_phi21G,e_phi21G.mask,phi31G,phi31G.mask,e_phi31G,e_phi31G.mask,NclEpG,NclEpBP,NclEpRP,NclEpRV,NclEpRV.mask,zpmagG,zpmagBP,zpmagBP.mask,zpmagRP,zpmagRP.mask,NHP1G,NHP1BP,NHP1RP,NHP1RV,NHP1RV.mask,_tab20_53,_tab20_54,_tab20_54.mask,_tab20_55,_tab20_55.mask,_tab20_56,_tab20_56.mask,FundFreq1,e_FundFreq1,FundFreq2,FundFreq2.mask,e_FundFreq2,e_FundFreq2.mask,Class,AG,AG.mask,e_AG,e_AG.mask,RA_ICRS,DE_ICRS,Norder,Dir,Npix
npartitions=12,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,double[pyarrow],double[pyarrow],int32[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],uint8[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],string[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],uint8[pyarrow],uint64[pyarrow],uint64[pyarrow]
288230376151711744,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170534137668829184,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

/ocean/projects/phy210048p/malanche/zubercal-filtering/cenv/lib/python3.12/site-packages/lsdb/dask/join_catalog_data.py:332: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,_RAJ2000,_DEJ2000,recno,Harm,Source,SolID,PF,PF.mask,e_PF,e_PF.mask,P1O,P1O.mask,e_P1O,e_P1O.mask,_tab20_8,e_EpochG,_tab20_10,_tab20_10.mask,e_EpochBP,e_EpochBP.mask,_tab20_12,_tab20_12.mask,e_EpochRP,e_EpochRP.mask,_tab20_14,_tab20_14.mask,e_EpochRV,e_EpochRV.mask,Gmagavg,e_Gmagavg,BPmagavg,BPmagavg.mask,e_BPmagavg,e_BPmagavg.mask,RPmagavg,RPmagavg.mask,e_RPmagavg,e_RPmagavg.mask,RVavg,RVavg.mask,e_RVavg,e_RVavg.mask,ptpG,e_ptpG,ptpBP,ptpBP.mask,e_ptpBP,e_ptpBP.mask,ptpRP,ptpRP.mask,e_ptpRP,e_ptpRP.mask,ptpRV,ptpRV.mask,e_ptpRV,e_ptpRV.mask,__M_H_,__M_H_.mask,e__M_H_,e__M_H_.mask,R21G,R21G.mask,e_R21G,e_R21G.mask,R31G,R31G.mask,e_R31G,e_R31G.mask,phi21G,phi21G.mask,e_phi21G,e_phi21G.mask,phi31G,phi31G.mask,e_phi31G,e_phi31G.mask,NclEpG,NclEpBP,NclEpRP,NclEpRV,NclEpRV.mask,zpmagG,zpmagBP,zpmagBP.mask,zpmagRP,zpmagRP.mask,NHP1G,NHP1BP,NHP1RP,NHP1RV,NHP1RV.mask,_tab20_53,_tab20_54,_tab20_54.mask,_tab20_55,_tab20_55.mask,_tab20_56,_tab20_56.mask,FundFreq1,e_FundFreq1,FundFreq2,FundFreq2.mask,e_FundFreq2,e_FundFreq2.mask,Class,AG,AG.mask,e_AG,e_AG.mask,RA_ICRS,DE_ICRS,Norder,Dir,Npix,objID,raMean,decMean,gMeanPSFMag,gMeanPSFMagErr,gFlags,rMeanPSFMag,rMeanPSFMagErr,rFlags,iMeanPSFMag,iMeanPSFMagErr,iFlags,zMeanPSFMag,zMeanPSFMagErr,zFlags,yMeanPSFMag,yMeanPSFMagErr,yFlags,_dist_arcsec,lc
npartitions=44238,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,double[pyarrow],double[pyarrow],int32[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],uint8[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],string[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],uint8[pyarrow],uint64[pyarrow],uint64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],int32[pyarrow],double[pyarrow],"nested<mjd: [double], mag: [float], magerr: [u..."
281474976710656,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

CPU times: user 6min 27s, sys: 13.2 s, total: 6min 40s
Wall time: 6min 51s


In [ ]:
%%time

with Client(n_workers=8, threads_per_worker=1) as client:
    display(client)
    matched.to_hats(LOCAL_HATS_PATH / f"zubercal_{CATALOG_TYPE}")
    # df = matched.compute()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 492.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38929,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 492.19 GiB
Comm: tcp://127.0.0.1:33177,Total threads: 1
Dashboard: http://127.0.0.1:38451/status,Memory: 61.52 GiB
Nanny: tcp://127.0.0.1:42901,
